In [1]:
#------------------------------------------------------------------------------
#
#------------------------------------------------------------------------------
#Reference:
# http://deap.readthedocs.io/en/master/examples/pso_basic.html
#------------------------------------------------------------------------------  

In [2]:
from deap import base, creator, tools, benchmarks
import random
import operator
import numpy

In [3]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, smin=None, smax=None, best=None)

In [4]:
#The initialization consists in generating a random position and a random speed for a particle. 
def generate(size, pmin, pmax, smin, smax):
    part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size)) 
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    part.smin = smin
    part.smax = smax
    return part

In [5]:
# The function creates a particle and initializes its attributes, 
# except for the attribute best, which will be set only after evaluation
def updateParticle(part, best, phi1, phi2):
    u1 = (random.uniform(0, phi1) for _ in range(len(part)))
    u2 = (random.uniform(0, phi2) for _ in range(len(part)))
    v_u1 = map(operator.mul, u1, map(operator.sub, part.best, part))
    v_u2 = map(operator.mul, u2, map(operator.sub, best, part))
    part.speed = list(map(operator.add, part.speed, map(operator.add, v_u1, v_u2)))
    for i, speed in enumerate(part.speed):
        if speed < part.smin:
            part.speed[i] = part.smin
        elif speed > part.smax:
            part.speed[i] = part.smax
    part[:] = list(map(operator.add, part, part.speed))

In [6]:
# register: individual (is assigned to the helper function)
# create class individuals with attr_bool for n=100 times. It means that 100 genes (0,1) are generated as a chromosome
# a chromosome represents a potential soluation of a problem
toolbox = base.Toolbox()
toolbox.register("particle", generate, size=2, pmin=-6, pmax=6, smin=-3, smax=3)
toolbox.register("population", tools.initRepeat, list, toolbox.particle)
toolbox.register("update", updateParticle, phi1=3.0, phi2=1.0)

In [7]:
# http://deap.readthedocs.io/en/master/api/benchmarks.html#deap.benchmarks.h1
toolbox.register("evaluate", benchmarks.h1)

In [8]:
# http://deap.readthedocs.io/en/master/api/benchmarks.html#deap.benchmarks.h1
#toolbox.register("evaluate", evalOneMax)

<figure>
  <img
  src="http://deap.readthedocs.io/en/master/_images/h1.png"
  alt="deap.benchmarks.h1"/>
</figure>


In [9]:
def main():   
    pop = toolbox.population(n=100)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = ["gen", "evals"] + stats.fields

    GEN = 1000
    best = None

    for g in range(GEN):
        for part in pop:
            part.fitness.values = toolbox.evaluate(part)
            if not part.best or part.best.fitness < part.fitness:
                part.best = creator.Particle(part)
                part.best.fitness.values = part.fitness.values
            if not best or best.fitness < part.fitness:
                best = creator.Particle(part)
                best.fitness.values = part.fitness.values
        for part in pop:
            toolbox.update(part, best)

        # Gather all the fitnesses in one list and print the stats
        logbook.record(gen=g, evals=len(pop), **stats.compile(pop))
        print(logbook.stream)
    
    return pop, logbook, best

In [10]:
if __name__ == "__main__":
    main()

gen	evals	avg      	std      	min       	max     
0  	100  	0.0861821	0.0537893	0.00637643	0.246499
1  	100  	0.094884 	0.0494186	0.00467902	0.226117
2  	100  	0.117932 	0.0820253	1.16487e-05	0.639054
3  	100  	0.178339 	0.178678 	0.0118657  	1.16312 
4  	100  	0.215897 	0.172674 	0.00184778 	0.967782
5  	100  	0.258909 	0.212123 	0.00104575 	1.26215 
6  	100  	0.281633 	0.282368 	0.00141883 	1.68324 
7  	100  	0.291181 	0.284097 	0.01226    	1.69199 
8  	100  	0.355456 	0.283466 	0.00878047 	1.46136 
9  	100  	0.29953  	0.233542 	0.0115272  	1.23099 
10 	100  	0.329777 	0.317307 	0.00611012 	1.43864 
11 	100  	0.27059  	0.203664 	0.00548795 	1.37159 
12 	100  	0.37833  	0.322769 	0.00606131 	1.96223 
13 	100  	0.352201 	0.289011 	0.00390325 	1.60802 
14 	100  	0.377428 	0.381856 	0.0119193  	1.78077 
15 	100  	0.33987  	0.298313 	0.00819687 	1.82192 
16 	100  	0.370747 	0.307746 	0.0107939  	1.91928 
17 	100  	0.349623 	0.310239 	0.00129303 	1.51566 
18 	100  	0.386012 	0.30512  	0.00